# Concurrent 패키지
* https://docs.python.org/3/library/concurrent.futures.html
* http://masnun.com/2016/03/29/python-a-quick-introduction-to-the-concurrent-futures-module.html


* v3.2부터 구현
* 스레드를 이용한 asynchronous execution: `ThreadPoolExecutor`
* 별도 프로세스를 이용한 asynchronous execution: `ProcessPoolExecutor`
* 둘 다 `Executor` 클래스에 같은 인터페이스로 정의됨

## `class Executor`
- 직접 사용되지 않고 subclass 구현을 통해 활용됨
- `ThreadPoolExecutor`와 `ProcessPoolExecutor` 두 서브클래스 존재


- __`submit(fn, *args, **kargs)`__
  - `fn(*args, **kargs)`을 실행되도록 스케줄링
  - `Future` 객체 반환
- __`map(func, *iterables, timeout=None, chunksize=1)`__
  - `map(func, *iterables)`와 같으나 asynchronous하게 실행된다는 점과 `func`이 concurrent하게 실행된다는 점이 다름
- __`shutdown(wait=True)`__
  - executor에게 사용 중인 자원을 내어놓도록 signal을 보냄

## `class ThreadPoolExecutor(max_workers=None)`
- `max_workers`는 (프로세서 개수*5)로 기본값
- 데드락 발생 가능성 주의
----
```python
import time
def wait_on_b():
    time.sleep(5)
    print(b.result())  # a를 기다리느라 b는 완료되지 못함
    return 5

def wait_on_a():
    time.sleep(5)
    print(a.result())  # b를 기다리느라 a는 완료되지 못함
    return 6


executor = ThreadPoolExecutor(max_workers=2)
a = executor.submit(wait_on_b)
b = executor.submit(wait_on_a)```

----
```python
def wait_on_future():
    f = executor.submit(pow, 5, 2)
    # 스레드는 하나만 생성가능한데, 이 함수를 또 실행하려고 하므로
    # 이 함수는 실행되지 못함
    print(f.result())

executor = ThreadPoolExecutor(max_workers=1)
executor.submit(wait_on_future)```
____

### `ThreadPoolExecutor` 예제

In [1]:
%%time
from concurrent.futures import ThreadPoolExecutor
from time import sleep
 
def return_after_5_secs(message):
    sleep(5)
    return message


pool = ThreadPoolExecutor(3)
future = pool.submit(return_after_5_secs, ("hello"))
print('submit 직후: ', future.done())

sleep(5)

print('5초 지난 후: ', future.done())
print('결과: ', future.result())

submit 직후:  False
5초 지난 후:  True
결과:  hello
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 5.01 s


In [2]:
import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://some-made-up-domain.com/']

# 하나의 페이지를 추출하여, URL과 내용을 반환
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# with문을 사용하여 스레드가 즉시 끝내도록 함
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

'http://www.foxnews.com/' page is 225141 bytes
'http://some-made-up-domain.com/' generated an exception: <urlopen error [Errno -2] Name or service not known>
'http://www.cnn.com/' page is 154947 bytes
'http://europe.wsj.com/' page is 911144 bytes
'http://www.bbc.co.uk/' page is 274394 bytes


## `class ProcessPoolExecutor(max_workers=None)`
- interactive 불가능
- `max_workers` 기본값은 프로세서 개수

### `ProcessPoolExcecutor` 예제

In [3]:
%%time
from concurrent.futures import ProcessPoolExecutor
from time import sleep
 
def return_after_5_secs(message):
    sleep(5)
    return message
 
pool = ProcessPoolExecutor(3)
 
future = pool.submit(return_after_5_secs, ("hello"))
print('submit 직후: ', future.done())

sleep(5)

print('5초 지난 후: ', future.done())
print('결과: ', future.result())

submit 직후:  False
5초 지난 후:  False
결과:  hello
CPU times: user 4 ms, sys: 20 ms, total: 24 ms
Wall time: 5.02 s


In [4]:
import concurrent.futures
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # executor.map()
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

if __name__ == '__main__':
    main()

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False


## `Future` 객체
* 호출가능한 것들(예: 함수)의 asynchronous execution을 은닉화
* `Executor.submit()`으로 생성


* __`cancel()`__
* __`cancelled()`__
* __`running()`__
* __`done()`__
* __`result(timeout=None)`__
* __`exception(timeout=None)`__
* __`add_done_callback(fn)`__


* __`set_running_or_notify_cancle()`__
* __`set_result(result)`__
* __ `set_exception(exception)`__

## 모듈 함수
* `concurrent.futures.`__`as_completed(fs, timeout=None)`__
  - `map()`은 전달하는 iterable 순서대로 결과를 반환하는 반면,
  - `as_completed()`는 완료되는 순서대로 결과를 반환
* `concurrent.futures.`__`wait(fs, timeout=None, return_when=ALL_COMPLETED)`__
  - tuple 반환 : (완료된 future들(결과 또는 예외), 완료되지 않은 future들)
  - `return_when`값은 `FIRST_COMPLETED`, `FIRST_EXCEPTION`, `ALL_COMPLETED` 중 하나

In [5]:
from concurrent.futures import ThreadPoolExecutor, wait, as_completed
from time import sleep
from random import randint

def return_after_5_secs(num):
    sleep(randint(1, 5))
    return "Return of {}".format(num)

pool = ThreadPoolExecutor(5)
futures = []
for x in range(5):
    futures.append(pool.submit(return_after_5_secs, x))

# concurrent.futures.as_completed()
for x in as_completed(futures):
    print(x.result())

Return of 2
Return of 3
Return of 0
Return of 1
Return of 4


In [6]:
from concurrent.futures import ThreadPoolExecutor, wait, as_completed
from time import sleep
from random import randint

def return_after_5_secs(num):
    sleep(randint(1, 5))
    return "Return of {}".format(num)

pool = ThreadPoolExecutor(5)
futures = []
for x in range(5):
    futures.append(pool.submit(return_after_5_secs, x))

# wait()
print(wait(futures, return_when='FIRST_COMPLETED'))

DoneAndNotDoneFutures(done={<Future at 0x7fb61851a518 state=finished returned str>}, not_done={<Future at 0x7fb618d78c88 state=running>, <Future at 0x7fb618d78828 state=running>, <Future at 0x7fb61851aac8 state=running>, <Future at 0x7fb62003c978 state=running>})
